In [ ]:
import pandas as pd

#Reading training data
data = pd.read_csv('train.csv')

In [ ]:
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,77515,"Please go on. well, i have a eatproblem Can yo...",1,0,0,0,0,0
1,76333,How long have you been on the verge of suicide...,1,1,0,0,0,0
2,60920,You are being a bit negative. I hate you Do yo...,1,0,0,0,0,0
3,76758,Will you be travelling in Economy Class? econo...,1,0,0,0,0,0
4,34060,Will you be travelling in Economy Class? No Wh...,1,0,0,0,0,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163072 entries, 0 to 163071
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             163072 non-null  object
 1   comment_text   163072 non-null  object
 2   toxic          163072 non-null  int64 
 3   severe_toxic   163072 non-null  int64 
 4   obscene        163072 non-null  int64 
 5   threat         163072 non-null  int64 
 6   insult         163072 non-null  int64 
 7   identity_hate  163072 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 10.0+ MB


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 6.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import copy
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import emoji

# BaseTokenizer class is an abstract class defining the structure for tokenizers
class BaseTokenizer(object):
    def process_text(self, text):
        raise NotImplemented

    def process(self, texts):
        for text in texts:
            yield self.process_text(text)

# Dictionary of regex patterns to replace common offensive words/phrases
RE_PATTERNS = {
    ' american ': ['amerikan'],
    ' adolf ': ['adolf'],
    ' hitler ': ['hitler'],
    ' fuck': [
        '(f)(u|[^a-z0-9 ])(c|[^a-z0-9 ])(k|[^a-z0-9 ])([^ ])*',
        '(f)([^a-z]*)(u)([^a-z]*)(c)([^a-z]*)(k)',
        ' f[!@#\$%\^\&\*]*u[!@#\$%\^&\*]*k', 'f u u c',
        '(f)(c|[^a-z ])(u|[^a-z ])(k)', r'f\*',
        'feck ', ' fux ', 'f\*\*',
        'f\-ing', 'f\.u\.', 'f###', ' fu ', 'f@ck', 'f u c k', 'f uck', 'f ck'
    ],
    ' ass ': [
        '[^a-z]ass ', '[^a-z]azz ', 'arrse', ' arse ', '@\$\$'
        '[^a-z]anus', ' a\*s\*s', '[^a-z]ass[^a-z ]',
        'a[@#\$%\^&\*][@#\$%\^&\*]', '[^a-z]anal ', 'a s s'
    ],
    ' ass hole ': [' a[s|z]*wipe', 'a[s|z]*[w]*h[o|0]+[l]*e', '@\$\$hole'],
    ' bitch ': [
        'b[w]*i[t]*ch', 'b!tch', 'bi\+ch', 'b!\+ch',
        '(b)([^a-z]*)(i)([^a-z]*)(t)([^a-z]*)(c)([^a-z]*)(h)',
        'biatch', 'bi\*\*h', 'bytch', 'b i t c h'
    ],
    ' bastard ': ['ba[s|z]+t[e|a]+rd'],
    ' trans gender': ['transgender'],
    ' gay ': ['gay'],
    ' cock ': [
        '[^a-z]cock', 'c0ck', '[^a-z]cok ', 'c0k', '[^a-z]cok[^aeiou]', ' cawk',
        '(c)([^a-z ])(o)([^a-z ]*)(c)([^a-z ]*)(k)', 'c o c k'
    ],
    ' dick ': [' dick[^aeiou]', 'deek', 'd i c k'],
    ' suck ': ['sucker', '(s)([^a-z ]*)(u)([^a-z ]*)(c)([^a-z ]*)(k)', 'sucks', '5uck', 's u c k'],
    ' cunt ': ['cunt', 'c u n t'],
    ' bull shit ': ['bullsh\*t', 'bull\$hit'],
    ' homo sex ual': ['homosexual'],
    ' jerk ': ['jerk'],
    ' idiot ': [
        'i[d]+io[t]+', '(i)([^a-z ]*)(d)([^a-z ]*)(i)([^a-z ]*)(o)([^a-z ]*)(t)', 'idiots',
        'i d i o t'
    ],
    ' dumb ': ['(d)([^a-z ]*)(u)([^a-z ]*)(m)([^a-z ]*)(b)'],
    ' shit ': [
        'shitty', '(s)([^a-z ]*)(h)([^a-z ]*)(i)([^a-z ]*)(t)', 'shite', '\$hit', 's h i t'
    ],
    ' shit hole ': ['shythole'],
    ' retard ': ['returd', 'retad', 'retard', 'wiktard', 'wikitud'],
    ' rape ': [' raped'],
    ' dumb ass': ['dumbass', 'dubass'],
    ' ass head': ['butthead'],
    ' sex ': ['sexy', 's3x', 'sexuality'],
    ' nigger ': ['nigger', 'ni[g]+a', ' nigr ', 'negrito', 'niguh', 'n3gr', 'n i g g e r'],
    ' shut the fuck up': ['stfu'],
    ' pussy ': ['pussy[^c]', 'pusy', 'pussi[^l]', 'pusses'],
    ' faggot ': [
        'faggot', ' fa[g]+[s]*[^a-z ]', 'fagot', 'f a g g o t', 'faggit',
        '(f)([^a-z ]*)(a)([^a-z ]*)([g]+)([^a-z ]*)(o)([^a-z ]*)(t)', 'fau[g]+ot', 'fae[g]+ot'
    ],
    ' mother fucker': [
        ' motha ', ' motha f', ' mother f', 'motherucker'
    ],
    ' whore ': ['wh\*\*\*', 'w h o r e']
}

# Tokenizer class that extends BaseTokenizer
class PatternTokenizer(BaseTokenizer):
    def __init__(self, lower=True, initial_filters=r"[^a-z0-9!@#\$%\^\&\*_\-,\.' ]", patterns=RE_PATTERNS,
                 remove_repetitions=True, convert_emoji=True):
        self.lower = lower  # To convert text to lowercase
        self.patterns = patterns  # Patterns for offensive words/phrases
        self.initial_filters = initial_filters  # Regex for initial filtering of text
        self.remove_repetitions = remove_repetitions  # To remove repeated characters
        self.stop_words = set(stopwords.words('english'))  # Stopwords for tokenization
        self.lemmatizer = WordNetLemmatizer()  # Lemmatizer for reducing words to their base form
        self.convert_emoji = convert_emoji  # To convert emojis to text

    def process_text(self, text):
        text = self._preprocess(text)  # Preprocess the text (e.g., remove HTML, URLs)
        for target, patterns in self.patterns.items():
            for pat in patterns:
                text = re.sub(pat, target, text)  # Replace offensive words/phrases
        text = re.sub(r"[^a-z' ]", ' ', text)  # Remove non-alphabetic characters
        tokens = word_tokenize(text)  # Tokenize the text
        tokens = [self.lemmatizer.lemmatize(token) for token in tokens if token not in self.stop_words]
        return tokens

    def process_ds(self, ds):
        ds = copy.deepcopy(ds)  # Deep copy of the dataset to avoid modifying the original
        if self.lower:
            ds = ds.str.lower()  # Convert to lowercase if specified
        if self.initial_filters is not None:
            ds = ds.str.replace(self.initial_filters, ' ', regex=True)  # Initial filtering of text
        if self.remove_repetitions:
            pattern = re.compile(r"(.)\1{2,}", re.DOTALL)
            ds = ds.str.replace(pattern, r"\1", regex=True)  # Remove repeated characters

        for target, patterns in self.patterns.items():
            for pat in patterns:
                ds = ds.str.replace(pat, target, regex=True)  # Replace offensive words/phrases

        ds = ds.str.replace(r"[^a-z' ]", ' ', regex=True)  # Remove non-alphabetic characters
        ds = ds.apply(self._tokenize_and_lemmatize)  # Tokenize and lemmatize the text
        return ds

    def _preprocess(self, text):
        text = BeautifulSoup(text, "html.parser").get_text()  # Remove HTML tags
        text = re.sub(r'http\S+', '', text)  # Remove URLs
        text = re.sub(r'@\w+', '', text)  # Remove mentions
        text = re.sub(r'#\w+', '', text)  # Remove hashtags
        if self.convert_emoji:
            text = self._convert_emojis(text)  # Convert emojis to text if specified
        if self.lower:
            text = text.lower()  # Convert to lowercase if specified
        if self.initial_filters is not None:
            text = re.sub(self.initial_filters, ' ', text)  # Initial filtering of text
        if self.remove_repetitions:
            pattern = re.compile(r"(.)\1{2,}", re.DOTALL)
            text = pattern.sub(r"\1", text)  # Remove repeated characters
        return text

    def _tokenize_and_lemmatize(self, text):
        tokens = word_tokenize(text)  # Tokenize the text
        tokens = [self.lemmatizer.lemmatize(token) for token in tokens if token not in self.stop_words]
        return ' '.join(tokens)  # Join tokens into a single string

    def _convert_emojis(self, text):
        return emoji.demojize(text)  # Convert emojis to descriptive text

def main():
    # Loading train and test datasets
    train = pd.read_csv("/content/train.csv")
    test = pd.read_csv("/content/test.csv")

    # Initialize tokenizer
    tokenizer = PatternTokenizer(convert_emoji=True)

    # Preprocess comment_text in train and test datasets
    train["comment_text"] = tokenizer.process_ds(train["comment_text"])
    test["comment_text"] = tokenizer.process_ds(test["comment_text"])

    # Save preprocessed data to CSV files
    train_output_path = "/content/train_preprocessed.csv"
    test_output_path = "/content/test_preprocessed.csv"

    train.to_csv(train_output_path, index=False)
    test.to_csv(test_output_path, index=False)

if __name__ == "__main__":
    main()


In [ ]:
train_df = pd.read_csv('train_preprocessed.csv')
test_df = pd.read_csv('test_preprocessed.csv')

In [ ]:
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,77515,please go well eatproblem elaborate yes e l z,1,0,0,0,0,0
1,76333,long verge suicide since year old tell year ol...,1,1,0,0,0,0
2,60920,bit negative hate wish hate yea e l z,1,0,0,0,0,0
3,76758,travelling economy class economy city flying i...,1,0,0,0,0,0
4,34060,travelling economy class city flying bangalore...,1,0,0,0,0,0


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163072 entries, 0 to 163071
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             163072 non-null  object
 1   comment_text   162972 non-null  object
 2   toxic          163072 non-null  int64 
 3   severe_toxic   163072 non-null  int64 
 4   obscene        163072 non-null  int64 
 5   threat         163072 non-null  int64 
 6   insult         163072 non-null  int64 
 7   identity_hate  163072 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 10.0+ MB


In [ ]:
test_df.head()

,id,comment_text
0,588793,yes indeed sort reminds elder lady played part...
1,369895,trans woman reading tweet right beautiful
2,716668,question broad criticize america country flee ...
3,442272,time illegals go back country origin keep free...
4,457562,starter bend one pink kick as pussy get taste ...


In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135556 entries, 0 to 135555
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            135556 non-null  int64 
 1   comment_text  135555 non-null  object
dtypes: int64(1), object(1)
memory usage: 2.1+ MB


In [ ]:
from sklearn.model_selection import train_test_split

# Splitting the data into training and validation sets
X = train_df['comment_text']
y = train_df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Filling NaN values with empty strings
X_train = X_train.fillna("")
X_val = X_val.fillna("")
test_df['comment_text'] = test_df['comment_text'].fillna("")

# Converting text data into TF-IDF features
vectorizer = TfidfVectorizer(max_features=10000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)
X_test_tfidf = vectorizer.transform(test_df['comment_text'])



In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import roc_auc_score
import numpy as np

# Defining the logistic regression model with L2 regularization
log_reg = LogisticRegression(max_iter=1000)

# Defining the parameter grid for GridSearchCV
param_grid = {'estimator__C': [0.01, 0.1, 1, 10, 100]}

# Creating the MultiOutputClassifier
multi_target_model = MultiOutputClassifier(log_reg, n_jobs=-1)

# Performing GridSearchCV to find the best regularization parameter
grid_search = GridSearchCV(multi_target_model, param_grid, cv=3, scoring='roc_auc', verbose=2)
grid_search.fit(X_train_tfidf, y_train)

# Get the best estimator
best_model = grid_search.best_estimator_

# Predicting probabilities for the validation data
y_val_pred = best_model.predict_proba(X_val_tfidf)

# Converting prediction from list of arrays to a single array
y_val_pred = np.array([pred[:, 1] for pred in y_val_pred]).T


Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] END ..................................estimator__C=0.01; total time=   4.1s
[CV] END ..................................estimator__C=0.01; total time=   3.2s
[CV] END ..................................estimator__C=0.01; total time=   3.1s
[CV] END ...................................estimator__C=0.1; total time=   3.4s
[CV] END ...................................estimator__C=0.1; total time=   3.3s
[CV] END ...................................estimator__C=0.1; total time=   3.6s
[CV] END .....................................estimator__C=1; total time=   7.9s
[CV] END .....................................estimator__C=1; total time=   6.1s
[CV] END .....................................estimator__C=1; total time=   8.9s
[CV] END ....................................estimator__C=10; total time=  16.1s
[CV] END ....................................estimator__C=10; total time=  17.0s
[CV] END ....................................esti

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Applying thresholding to get binary predictions (threshold = 0.5)
threshold = 0.5
val_pred_binary = (y_val_pred >= threshold).astype(int)

# Evaluating the model using the validation set
roc_auc_val = roc_auc_score(y_val, y_val_pred, average='macro')
print(f'Validation ROC AUC: {roc_auc_val:.4f}')

accuracy = accuracy_score(y_val, val_pred_binary)
precision = precision_score(y_val, val_pred_binary, average='macro')
recall = recall_score(y_val, val_pred_binary, average='macro')
f1 = f1_score(y_val, val_pred_binary, average='macro')

print(f'Validation Accuracy: {accuracy:.4f}')
print(f'Validation Precision: {precision:.4f}')
print(f'Validation Recall: {recall:.4f}')
print(f'Validation F1 Score: {f1:.4f}')

Validation ROC AUC: 0.9745
Validation Accuracy: 0.9101
Validation Precision: 0.7084
Validation Recall: 0.4107
Validation F1 Score: 0.5026


In [ ]:
# Predicting probabilities for the test data
test_pred = best_model.predict_proba(X_test_tfidf)
test_pred = np.array([pred[:, 1] for pred in test_pred]).T


In [ ]:
# Saving the predictions
output_path = "/content/test_predictions.csv"
test_pred_df = pd.DataFrame(test_pred, columns=y_train.columns)
test_pred_df['comment_text'] = test_df['comment_text'].values


In [ ]:
test_pred_df.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate,comment_text
0,0.032944,0.005416,0.006737,0.003079,0.006721,0.008096,yes indeed sort reminds elder lady played part...
1,0.141763,0.014296,0.018479,0.003066,0.020952,0.008086,trans woman reading tweet right beautiful
2,0.380781,0.041821,0.094516,0.003267,0.084213,0.032276,question broad criticize america country flee ...
3,0.068636,0.012653,0.014659,0.004228,0.029865,0.008659,time illegals go back country origin keep free...
4,0.968883,0.049891,0.954708,0.023356,0.522066,0.019222,starter bend one pink kick as pussy get taste ...


In [ ]:
test_pred_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135556 entries, 0 to 135555
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   toxic          135556 non-null  float64
 1   severe_toxic   135556 non-null  float64
 2   obscene        135556 non-null  float64
 3   threat         135556 non-null  float64
 4   insult         135556 non-null  float64
 5   identity_hate  135556 non-null  float64
 6   comment_text   135556 non-null  object 
dtypes: float64(6), object(1)
memory usage: 7.2+ MB


In [ ]:
test_pred_df.to_csv(output_path, index=False)
print(f'Test predictions saved to {output_path}')

Test predictions saved to /content/test_predictions.csv
